In [1]:
import pandas as pd
import numpy as np

In [2]:
times = pd.read_csv('timesData.csv')
shangai = pd.read_csv('shanghaiData.csv')
cwur = pd.read_csv('cwurData.csv')

In [23]:
rank1 = pd.DataFrame(cwur.groupby('institution').count()['year'])
rank1['cwur'] = 1

rank2 = pd.DataFrame(shangai.groupby('university_name').count()['year'])
rank2['shangai'] = 1

rank3 = pd.DataFrame(times.groupby('university_name').count()['year'])
rank3['times'] = 1

In [27]:
outjoi = pd.concat([rank1, rank2, rank3],axis = 1, sort = True)
outjoi = outjoi.replace(np.nan, 0)

df10 = pd.DataFrame(columns = ['num_ranks'])
df10['num_ranks'] = outjoi['cwur'] + outjoi['shangai'] + outjoi['times']
df10

,num_ranks
AGH University of Science and Technology,2.0
Aalborg University,3.0
Aalto University,3.0
Aarhus University,3.0
Aberystwyth University,2.0
...,...
École centrale de Lyon,1.0
École normale supérieure - Paris,1.0
École normale supérieure de Cachan,1.0
École normale supérieure de Lyon,1.0


In [28]:
#POTENTIAL MISMATCHES
pm = outjoi[['cwur', 'shangai', 'times']]
pm['num_ranks'] = pm['cwur'] + pm['shangai'] + pm['times']
pm = pm[pm['num_ranks'] == 1]
pm

C:\Users\sabrm\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,cwur,shangai,times,num_ranks
Adam Mickiewicz University,0.0,0.0,1.0,1.0
Adam Mickiewicz University in Poznań,1.0,0.0,0.0,1.0
Ain Shams University,1.0,0.0,0.0,1.0
Aix Marseille University,0.0,1.0,0.0,1.0
Albany Medical College,1.0,0.0,0.0,1.0
...,...,...,...,...
École Polytechnique de Montréal,1.0,0.0,0.0,1.0
École centrale de Lyon,1.0,0.0,0.0,1.0
École normale supérieure - Paris,1.0,0.0,0.0,1.0
École normale supérieure de Cachan,1.0,0.0,0.0,1.0


In [29]:
mism_cw = pm[pm['cwur'] == 1]
mism_cw = pd.DataFrame(mism_cw['cwur'])
mism_cw.reset_index(inplace = True)
mism_cw = mism_cw.rename(columns = {'index':'uni'})
mism_cw

,uni,cwur
0,Adam Mickiewicz University in Poznań,1.0
1,Ain Shams University,1.0
2,Albany Medical College,1.0
3,Albert Ludwig University of Freiburg,1.0
4,"All India Institute of Medical Sciences, New D...",1.0
...,...,...
355,École Polytechnique de Montréal,1.0
356,École centrale de Lyon,1.0
357,École normale supérieure - Paris,1.0
358,École normale supérieure de Cachan,1.0


In [35]:
mism_sh = pm[pm['shangai'] == 1]
mism_sh = pd.DataFrame(mism_sh['shangai'])
mism_sh.reset_index(inplace = True)
mism_sh = mism_sh.rename(columns = {'index':'uni'})
mism_sh

,uni,shangai
0,Aix Marseille University,1.0
1,Arizona State University - Tempe,1.0
2,Beijing University of Aeronautics and Astronau...,1.0
3,Capital University of Medical Sciences,1.0
4,Catholic University of Chile,1.0
...,...,...
194,Vrije Universiteit Brussel (VUB),1.0
195,Vrije University Brussel,1.0
196,Washington State University - Pullman,1.0
197,Western University,1.0


In [36]:
mism_tm = pm[pm['times'] == 1]
mism_tm = pd.DataFrame(mism_tm['times'])
mism_tm.reset_index(inplace = True)
mism_tm = mism_tm.rename(columns = {'index':'uni'})
mism_tm

,uni,times
0,Adam Mickiewicz University,1.0
1,Alexandru Ioan Cuza University,1.0
2,Aligarh Muslim University,1.0
3,American University,1.0
4,American University of Sharjah,1.0
...,...,...
202,Xi’an Jiaotong University,1.0
203,Yıldız Technical University,1.0
204,École Normale Supérieure,1.0
205,École Normale Supérieure de Lyon,1.0


https://towardsdatascience.com/string-matching-with-fuzzywuzzy-e982c61f8a84
<br> link to complete explanation

In [47]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [43]:
cw_uni = list(mism_cw.uni.unique())
sh_uni = list(mism_sh.uni.unique())
tm_uni = list(mism_tm.uni.unique())

In [108]:
#Defining a function to return the match and similarity score of the fuzz.ratio() scorer. The function will take in a term(name), list of terms(list_names), and a minimum similarity score(min_score) to return the match. 
def match_names(name, list_names, min_score=0):
    max_score = -1
    max_name = ''
    for x in list_names:
        score = fuzz.partial_ratio(name, x)
        if (score > min_score) & (score > max_score):
            max_name = x
            max_score = score
    return (max_name, max_score)

In [186]:
#For loop to create a list of tuples with the first value being the name from the second dataframe (name to replace) and the second value from the first dataframe (string replacing the name value). Then, casting the list of tuples as a dictionary. 
names = []
for x in cw_uni:
    match = match_names(x, tm_uni, 95)
    if match[1] >= 0:
        name = ('(' + str(x), str(match[0]) + ')')
        names.append(name)
name_dict_cw2tm = dict(names)
name_dict_cw2tm

{'(Adam Mickiewicz University in Poznań': 'Adam Mickiewicz University)',
 '(Babeș-Bolyai University': 'Babeş-Bolyai University)',
 '(Binghamton University': 'Binghamton University, State University of New York)',
 '(China Medical University (Taiwan)': 'China Medical University, Taiwan)',
 '(Federal University of Paraná': 'Federal University of Paraná (UFPR))',
 '(Indiana University - Bloomington': 'Indiana University)',
 '(Indiana University-Purdue University Indianapolis': 'Indiana University)',
 "(Julius-Maximilian's University of Würzburg": 'University of Würzburg)',
 '(National Research Nuclear University MEPhI': 'National Research Nuclear University MePhI)',
 '(National Taiwan University of Science and Technology': 'National Taiwan University of Science and Technology (Taiwan Tech))',
 '(Ohio State University, Columbus': 'Ohio State University)',
 '(Paris Diderot University - Paris 7': 'Paris Diderot University – Paris 7)',
 '(Pennsylvania State University, University Park': 'Penn

In [187]:
#For loop to create a list of tuples with the first value being the name from the second dataframe (name to replace) and the second value from the first dataframe (string replacing the name value). Then, casting the list of tuples as a dictionary. 
names = []
for x in sh_uni:
    match = match_names(x, tm_uni, 95)
    if match[1] >= 0:
        name = ('(' + str(x), str(match[0]) + ')')
        names.append(name)
name_dict_sh2tm = dict(names)
name_dict_sh2tm

{'(China Medical University': 'China Medical University, Taiwan)',
 '(Indiana University Bloomington': 'Indiana University)',
 '(Indiana University-Purdue University at Indianapolis': 'Indiana University)',
 '(Louisiana State University Health Sciences Center': 'Louisiana State University)',
 '(Paris Dauphine University (Paris 9)': 'Paris Dauphine University)',
 '(Pennsylvania State University - University Park': 'Pennsylvania State University)',
 '(Pierre and Marie  Curie University - Paris 6': 'Pierre and Marie Curie University)',
 '(Purdue University - West Lafayette': 'Purdue University)',
 '(Rutgers, The State University of New Jersey': 'Rutgers, the State University of New Jersey)',
 '(Rutgers, The State University of New Jersey - New Brunswick': 'Rutgers, the State University of New Jersey)',
 '(Sao Paulo State University': 'São Paulo State University (UNESP))',
 '(Swiss Federal Institute of Technology Zurich': 'ETH Zurich – Swiss Federal Institute of Technology Zurich)',
 '(Tec

In [189]:
#Using the dictionary to replace the keys with the values in the 'name' column for the second dataframe
mism_sh.uni = mism_sh.uni.replace(name_dict_sh2tm)
mism_sh

,uni,shangai
0,Aix Marseille University,1.0
1,Arizona State University - Tempe,1.0
2,Beijing University of Aeronautics and Astronau...,1.0
3,Capital University of Medical Sciences,1.0
4,Catholic University of Chile,1.0
...,...,...
194,Vrije Universiteit Brussel (VUB),1.0
195,Vrije University Brussel,1.0
196,Washington State University - Pullman,1.0
197,Western University,1.0


In [190]:
#Using the dictionary to replace the keys with the values in the 'name' column for the second dataframe
mism_cw.uni = mism_cw.uni.replace(name_dict_cw2tm)
mism_cw

,uni,cwur
0,Adam Mickiewicz University in Poznań,1.0
1,Ain Shams University,1.0
2,Albany Medical College,1.0
3,Albert Ludwig University of Freiburg,1.0
4,"All India Institute of Medical Sciences, New D...",1.0
...,...,...
355,École Polytechnique de Montréal,1.0
356,École centrale de Lyon,1.0
357,École normale supérieure - Paris,1.0
358,École normale supérieure de Cachan,1.0


In [193]:
cwsh = pd.merge(mism_cw, mism_sh, how='outer', on='uni')
cwsh

,uni,cwur,shangai
0,Adam Mickiewicz University in Poznań,1.0,NaN
1,Ain Shams University,1.0,NaN
2,Albany Medical College,1.0,NaN
3,Albert Ludwig University of Freiburg,1.0,NaN
4,"All India Institute of Medical Sciences, New D...",1.0,NaN
...,...,...,...
554,Vrije Universiteit Brussel (VUB),NaN,1.0
555,Vrije University Brussel,NaN,1.0
556,Washington State University - Pullman,NaN,1.0
557,Western University,NaN,1.0


In [194]:
cwshtm = pd.merge(cwsh, mism_tm, how='outer', on='uni')
cwshtm

,uni,cwur,shangai,times
0,Adam Mickiewicz University in Poznań,1.0,NaN,NaN
1,Ain Shams University,1.0,NaN,NaN
2,Albany Medical College,1.0,NaN,NaN
3,Albert Ludwig University of Freiburg,1.0,NaN,NaN
4,"All India Institute of Medical Sciences, New D...",1.0,NaN,NaN
...,...,...,...,...
761,Xi’an Jiaotong University,NaN,NaN,1.0
762,Yıldız Technical University,NaN,NaN,1.0
763,École Normale Supérieure,NaN,NaN,1.0
764,École Normale Supérieure de Lyon,NaN,NaN,1.0
